# Исследование надёжности заёмщиков

***Описание проекта***
  В рамках проекта необходимо проанализировать данные статистики о платежеспособности клиентов и оценить факторы влияющие на факт погашения кредита в срок

***Краткое описание данных, представленных в датасете***
   
   *Данные в датасете data*
   
    children — количество детей в семье
    days_employed — общий трудовой стаж в днях
    dob_years — возраст клиента в годах
    education — уровень образования клиента
    education_id — идентификатор уровня образования
    family_status — семейное положение
    family_status_id — идентификатор семейного положения
    gender — пол клиента
    income_type — тип занятости
    debt — имел ли задолженность по возврату кредитов
    total_income — ежемесячный доход
    purpose — цель получения кредита
    

***Содержание проекта***


1. <a href='#the_destination'> Предварительный анализ данных</a>

    1.1  <a href='#the_destination1'>Загрузка необходимых библиотек</a>

    1.2. <a href='#the_destination2'>Загрузка датасета и предварительный анализ</a>
    
    1.3 . <a href='#the_destination3'>Вывод по разделу 1</a>
    

2. <a href='#the_destination4'>Изучение и проверка данных</a>

    2.1  <a href='#the_destination5'>Обработка пропусков </a>

    2.1.1  <a href='#the_destination6'>Проверка наличия нулевых значений в различных столбцах</a>
    
    2.1.2. <a href='#the_destination7'>Корректировка значений столбца с количеством детей</a>
    
    2.1.3. <a href='#the_destination8'>Корректировка данных стобца дохода методом заполнения медианами по роду деятельности</a>

    2.1.4. <a href='#the_destination9'>Замена типов данных</a>
    
    2.1.5. <a href='#the_destination10'>Обработка дубликатов</a>
    
    2.1.6. <a href='#the_destination11'>Лемматизация</a>
    
    2.1.7. <a href='#the_destination12'>Категоризация данных</a>
    
    
3. <a href='#the_destination13'>Изучение влияния различных факторов на факт возврата кредита в срок</a>

    3.1. <a href='#the_destination14'>Влияние количества детей</a>
    
    3.2. <a href='#the_destination15'>Влияние семейного положения</a>
    
    3.3. <a href='#the_destination16'>Влияние уровня дохода</a>
    
    3.4. <a href='#the_destination17'>Влияние целей</a>


4. <a href='#the_destination18'>Общий вывод</a>

<a id='the_destination'></a>
# 1. Предварительный анализ данных

<a id='the_destination1'></a>
## 1.1. Загрузка необходимых библиотек

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
import collections

<a id='the_destination2'></a>
## 1.2. Загрузка датасета и предварительный анализ

In [2]:
clients_solvency = pd.read_csv('/datasets/data.csv')
# изучение информации о таблице
clients_solvency.info()
# печать первых 5 столбцов таблицы
display(clients_solvency.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
print('Изучение данных о безработных клиентов банка')
display(clients_solvency[clients_solvency['income_type'] == 'безработный'])

Изучение данных о безработных клиентов банка


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
14798,0,395302.838654,45,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью


In [4]:
print('Изучение данных о клиентах с сомнительной информацией о детях')
super_family = clients_solvency[clients_solvency['children'] > 10]
display(super_family.head(10))
print('Количество клиентов указавших количество детей в семье более 10: {} чел'.format(super_family.shape[0]))

Изучение данных о клиентах с сомнительной информацией о детях


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
3302,20,NaN,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,NaN,профильное образование
3396,20,NaN,56,высшее,0,женат / замужем,0,F,компаньон,0,NaN,высшее образование
3671,20,-913.161503,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,101255.492076,на покупку подержанного автомобиля
3697,20,-2907.910616,40,среднее,1,гражданский брак,1,M,сотрудник,0,115380.694664,на покупку подержанного автомобиля
3735,20,-805.044438,26,высшее,0,Не женат / не замужем,4,M,сотрудник,0,137200.646181,ремонт жилью


Количество клиентов указавших количество детей в семье более 10: 76 чел


In [5]:
print('Количество клиентов с сомнительным уровнем доходов')
print('Количество клиентов с стажем менее полгода {} чел - проверка положительных значений стажа'.format(clients_solvency[(clients_solvency['days_employed'] > 0) & (clients_solvency['days_employed'] < 140)]['dob_years'].count()))
print('Количество клиентов с стажем менее полгода {} чел - проверка отрицательных значений стажа'.format(clients_solvency[(clients_solvency['days_employed'] < 0) & (clients_solvency['days_employed'] > -140)]['dob_years'].count()))
doubtful_clients = clients_solvency[(clients_solvency['days_employed'] < 0) & (clients_solvency['days_employed'] > -140) & (clients_solvency['dob_years'] > 30) & (clients_solvency['total_income'] > 120000)]
display(doubtful_clients.head(10))
print('Количество клиентов, работавших менее полгода, в возрасте более 30 лет и с уровнем дохода более 120 тысяч: {} чел'.format(doubtful_clients.shape[0]))

Количество клиентов с сомнительным уровнем доходов
Количество клиентов с стажем менее полгода 0 чел - проверка положительных значений стажа
Количество клиентов с стажем менее полгода 358 чел - проверка отрицательных значений стажа


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
64,0,-118.891411,35,неоконченное высшее,2,Не женат / не замужем,4,M,сотрудник,0,282900.471308,сделка с подержанным автомобилем
180,0,-60.637328,37,среднее,1,гражданский брак,1,F,сотрудник,0,212257.212507,сыграть свадьбу
188,0,-99.195947,48,среднее,1,женат / замужем,0,M,сотрудник,0,173544.679523,покупка своего жилья
195,0,-117.127886,51,среднее,1,женат / замужем,0,F,сотрудник,0,255838.361680,дополнительное образование
215,0,-79.880350,42,высшее,0,Не женат / не замужем,4,F,компаньон,0,260465.394500,операции с недвижимостью
359,0,-133.876721,44,высшее,0,гражданский брак,1,F,компаньон,0,149071.526683,покупка жилья для сдачи
485,0,-114.616311,52,среднее,1,женат / замужем,0,F,компаньон,0,270686.003288,строительство недвижимости
569,0,-88.433928,46,среднее,1,женат / замужем,0,M,компаньон,0,121006.968749,дополнительное образование
613,0,-113.370407,53,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,126309.172566,высшее образование
656,0,-87.975919,49,среднее,1,женат / замужем,0,M,сотрудник,0,161791.974060,покупка жилья


Количество клиентов, работавших менее полгода, в возрасте более 30 лет и с уровнем дохода более 120 тысяч: 164 чел


<a id='the_destination3'></a>
## 1.3. Вывод по разделу 1

В таблице содержится информация о платежеспособности клиентов банка. В ходе анализа информации о таблице было выявлено, что  двух столбцах - в столбце с трудовым стажем (days_employed) и столбце с совокупным доходом (total_income) присутствуют пропуски данных. Такие пропуски данных могли возникнуть при вводе данных клиентом, оператором банка или при сведении данных в общую таблицу в результате программной ошибки. При выведении таблицы было принято решение вывести срез первых 15 строк для более внимательной визуальной оценки информации таблицы. 

Как показало изучение таблицы,  в столбце с трудовым стажем (days_employed) присутствуют отрицательные значения, что может быть связано с сохранившимся в некоторых местах дефисом или спецификой работы алгоритма расчета дней работы (для работающих стаж выражен в отрицательных значениях, для пенсионеров и безработных - положительным), также значения трудового стажа для некоторых ячеек не соответствуют реальности или вызывают сомнения (столбец 4 - 340266 дней работы соответствует 1308 годам работы, также стоит обратить внимание на значение в 8 - 6930 дней работы составляют 19 лет работы, что вызывает сомнение, так как в таком случае данный клиент работал официально с 16 лет). Вероятно, завышенные в разы данные связаны с ошибкой ввода данных, однако значения с малым разбросом могут указывать на неверную информацию, переданную клиентом. В столбце с данными об образовании (education) присутствуют различные написания уровней образования. В столбце с указанием цели кредита (purpose) присутствуют различные формулировки одних целей - "сыграть свадьбу" или "на проведение свадьбы", "покупка жилья" или "ппокупка жилой недвижимости". Также необходимо отметить, что при указании семейного статуса (family_status) указывается юридически неверный термин - "гражданский брак" вместо термина "сожительство". Необходимо осуществить обработку данных, привести данные к единому виду, принять решение о целесообразности удаления или сохранения пустых ячеек, скорректировать данные в столбцах с трудовым стажем. Также была изучена информация о безработных клиентах банках - данные о стаже данных клиентов нереалистичны, а указыванные уровни доходов вызывают сомнения, что должно быть учтено при дальнейшем анализе данных (при невозможности перепроверки данных, дданных клиентов целесообразно удалить из анализа). 

Также был осуществлен анализ наличия клиентов с сомнительными данными о детях, так как в России семьи с 10 и более детей составляют 0,07% от общего количества семей (данные с сайта https://oms.msk.ru/ "Объединение многодетных семей города Москвы"), то в данной таблице можно ожидать небольшое количества клиентов с количеством детей более 10 (не более 15 человек), однако в ходе анализа было найдено 76 человек с количеством детей более 10. Можно обратить внимание, что у большинства из них указано количество детей как 20, что говорит о вероятной ошибке оператора. При категоризации данных необходимо обратить внимание на данную ситуацию - выявить количество позиций вероятно относящимся к категории 1 и 2 ребенка (напечатанных в виде 10 и 20), а также выявить тех клиентов данные о количестве которых необходимо уточнить (семьи с 11 и более детей). Также были проанализированы данные о тех из клиентов, кто имеет стаж менее полугода (стаж указан в отрицательной форме, так как проверке было выявлено, что все подобные значения стажа имеют перед собой дефис), возраст более 30 лет и уровень дохода более 120 тысяч - сочетание таких данных было выбрано так как отражает ту часть людей, которые сталкиваются с проблемами с поиском работы при отсутствии стажа и не могут рассчитывать на высокий уровень стартовой зарплаты. Было обнаружено 164 человека, предоставившие высокие данные о доходах при незначительном стаже и возрасте более 30 лет. Информация, предоставленая данными клиентами вызывает сомнения и ее необходимо перепроверить и возможно исключить из окончательного анализа уровня доходов. 

<a id='the_destination4'></a>
# 2. Предварительная обработка данных

<a id='the_destination5'></a>
## 2.1. Обработка пропусков

<a id='the_destination6'></a>
## 2.1.1. Проверка наличия нулевых значений в различных столбцах

In [6]:
#проверка наличия нулевых значений в столбцах общего дохода
null_income_clients_number = clients_solvency['total_income'].isna().count()
print('Количество людей с нулевым значением дохода: {} чел'.format(null_income_clients_number))
#проверка наличия нулевых значений и значений менее 18 в столбце с возрастом
baby_clients_number = clients_solvency[clients_solvency['dob_years'] == 0]['total_income'].count()
print('Количество людей с нулевым значением возраста: {} чел'.format(baby_clients_number))
underage_clients_number = clients_solvency[(clients_solvency['dob_years'] < 18) & (clients_solvency['dob_years'] > 0)]['total_income'].count()
print('Количество людей с возрастом до 18 лет: {} чел'.format(underage_clients_number)) 

Количество людей с нулевым значением дохода: 21525 чел
Количество людей с нулевым значением возраста: 91 чел
Количество людей с возрастом до 18 лет: 0 чел


In [7]:
display(clients_solvency.head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В ходе обработки таблицы был проведен анализ количества людей с пропущенными значениями общего дохода (total_income), трудового стажа (days_employed). В столбце дохода (total_income) - было обнаружено 21525 с отсутствующей информацией о доходах. В  столбце с данными о возрасте клиентов банка, был обнаружен 91 человек с нулевым значением возраста и ни одного человека менее 18 лет, нулевые значения возраста говорят о вероятной ошибке оператора, вводившего информацию в базу данных, возможно приблизительно оценить возраст данных клиентов по их совокупному стажу(days_employed) и типу занятости (income_type), но это может привести к искажению данных(например, мы можем рассмотреть строки 7 и 8, при малом значении стажа возраст клиента составляет 50 лет, а у клиента №8 при значительном-лишь 35 лет, если рассматривать возраст с точки зрения занятости, то клиентка №4 в 53 года является пенсионером, а №12 в 56 - работающей), и так как в реализуемом нами анализе не стоит задача оценки влияния возраста на платежеспособность клиента, то данные значения целесообразно оставить без изменений, указав сотрудникам банка на необходимость уточнить данные по этим клиентам.

<a id='the_destination7'></a>
## 2.1.2. Корректировка заполнения столбца с информацией о детях

In [8]:
print('Анализ значений количества детей у клиентов банка')
print(clients_solvency['children'].value_counts())
# Замена значений количества детей
clients_solvency['children'] = clients_solvency['children'].replace(-1, 1).replace(20, 2)
print('Проверка корректировки значений таблицы')
print(clients_solvency['children'].value_counts())

Анализ значений количества детей у клиентов банка
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
Проверка корректировки значений таблицы
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


Были проанализированы значения столбца "количество детей"(children), был выведен общий список основных значений столбца. Среди значений столбца было обнаружено кроме 67 клиентов с 20 детьми, 44 клиента с значением "-1" в ячейке количества детей. Это явная ошибка оператора или программы переноса данных, наиболее целесообразно создать вторую таблицу, в которой заменить значения 20 на 2, и в дальнейшем будет проводить анализ двух таблиц, однако так как перед нами не не стоит оценки влияния количества детей на платежеспособность заемщика - то мы можем внести корректировки и заменить 20 на 2, не боясь потерять данные. 

<a id='the_destination8'></a>
## 2.1.3. Корректировка данных стобца дохода методом заполнения медианами по роду деятельности

In [9]:
# Определение медиан дохода по роду деятельности
# Создание массива уникальных значеней типа дохода
list_income_type = clients_solvency['income_type'].unique()

# Написание функции по определению медиан по роду деятельности 
def median(dataframe, list_of_jobs, name):
    list_of_median = []
    for i in range(len(list_of_jobs)):
        tulip = (list_of_jobs[i], dataframe[dataframe['income_type'] == list_of_jobs[i]][name].median())
        list_of_median.append(tulip)
    return list_of_median


# Создание списка с категориями деятельности, возрастом и доходом
list_of_median_income = median(clients_solvency, list_income_type, 'total_income')
list_of_median_days = median(clients_solvency, list_income_type, 'days_employed')
print(list_of_median_income)
print()
print(list_of_median_days)
print()

# заполнение пропущенных ячеек медианным значением
for i in range(len(list_of_median_income)):
    mask = (clients_solvency['income_type'] == list_of_median_income[i][0])
    clients_solvency.loc[mask, 'total_income'] = clients_solvency.loc[mask, 'total_income'].fillna(list_of_median_income[i][1])
    clients_solvency.loc[mask, 'days_employed'] = clients_solvency.loc[mask, 'days_employed'].fillna(list_of_median_days[i][1])

[('сотрудник', 142594.39684740017), ('пенсионер', 118514.48641164352), ('компаньон', 172357.95096577113), ('госслужащий', 150447.9352830068), ('безработный', 131339.7516762103), ('предприниматель', 499163.1449470857), ('студент', 98201.62531401133), ('в декрете', 53829.13072905995)]

[('сотрудник', -1574.2028211070851), ('пенсионер', 365213.30626573117), ('компаньон', -1547.3822226779334), ('госслужащий', -2689.3683533043886), ('безработный', 366413.65274420456), ('предприниматель', -520.8480834953765), ('студент', -578.7515535382181), ('в декрете', -3296.7599620220594)]



In [10]:
clients_solvency.info()

clients_solvency.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Были внесены изменения в список данных, для заполнения таблицы медианными значениями сначала был определен список уникальных значений категорий источников дохода. Затем была введена функция для создания списка с значением категории дохода и медианным значением параметра, данная функция была применена на значения параметров общего дохода и дней на работе.
Далее в цикле было осуществленно заполнение пропущенных значений медианными значениями при помощи функции fillna().

Была осуществленна проверка наличия пропущенных ячеек после обработки, пропущенные ячейки остутствуются. Для контроля правильности заполнения данных были выведены первые 15 строк таблицы, 12 строка таблицы до этого имевшая пропущенные ячейки заполнена в соответствии с медианными значениями.

<a id='the_destination9'></a>
## 2.1.4. Замена типа данных

In [11]:
# замена типа данных в столбце days_employed с float на int
clients_solvency['days_employed'] = clients_solvency['days_employed'].astype('int64')
# замена типа данных в столбцеtotal_income  с float на int
clients_solvency['total_income'] = clients_solvency['total_income'].astype('int64')
clients_solvency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [12]:
#оценка возможных причин появления завышенных значений
unemployed_clients = clients_solvency[clients_solvency['income_type'] == 'пенсионер']
short_list_unemployed = unemployed_clients[['days_employed', 'dob_years']]
display(short_list_unemployed.sort_values(by='days_employed', ascending=False).tail(5))
print(short_list_unemployed['days_employed'].value_counts())

#добавление столбца с кодом вида пенсии
#функция выделения кода
def code_adding(number):
    if number < 0:
        return 0
    else:
        return number
    
clients_solvency['unemployment_code'] = clients_solvency['days_employed'].apply(code_adding)
display(clients_solvency.head())

,days_employed,dob_years
7229,328827,32
14783,328795,62
17782,328771,56
9328,328734,41
20444,328728,72


365213    414
391277      2
358183      2
377209      2
342836      2
         ... 
347531      1
368009      1
390536      1
337286      1
374787      1
Name: days_employed, Length: 3368, dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,unemployment_code
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,0
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,0
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,0
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,340266


In [13]:
#функция перевода значений трудового стажа
#функция перевода числа в ноль или в число по модулю
def replace_days_for_unemployed(number):
    if number > 0:
        return 0
    else:
        return abs(number)

clients_solvency['days_employed'] = clients_solvency['days_employed'].apply(replace_days_for_unemployed)
display(clients_solvency.head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,unemployment_code
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,0
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,0
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,0
4,0,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,340266


После анализа информации о таблице с данными клиентов было выявлено, что столбцы с информацией о стаже (days_employed) и общем доходе (total_income) имеют тип данных float, однако, стаж и общий доход удобнее оценивать в виде целочисленных значений, поэтому была произведена замена типа данных на целочисленные значения методом .astype. Была осуществленна проверка вызовом функции info().
Также была проведена попытка анализа причин появления существенно завышенных данных о трудовом стаже для пенсионеров и безработных - для этого была сформирована таблица с значениями стажа и возраста для пенсионеров, были выведены значения стажа и возраста для пенсионеров и упорядочены по убыванию значения стажа. В этой таблице хорошо видно, что не наблюдается прямой связи между размером трудового стажа и возрастом клиента (значение 328827 соответствует клиенту №7229 32 лет, а значение 328728 - клиенту №20444 72 лет), таким образом можно сделать вывод, что это вероятнее всего это коды из Общероссийского классификатора информации по социальной защите населения, где указаны причины и виды начисления пенсии. На это указывает и повторение одинаковых значений трудового стажа у разных клиентов, которое было оценено при помощи вызова функции value_counts. Поэтому было принято решение заменить численную запись стажа на 0, а в таблицу ввести дополнительный столбец с кодом стажа (unempoyment_code). Также отрицательные значения стажа были заменены на положительную запись. Таблица стала более ясной и понятной для чтения. 

<a id='the_destination10'></a>
## 2.1.5. Обработка дубликатов

### 2.1.5.1 Проверка на полностью повторяющиеся строки

In [14]:
#проверка на повторяющиеся полностью строки в таблице
print('Количество полностью дублирующихся строк в таблице:', clients_solvency.duplicated().sum())
#определение уникальных значений в столбцах и приведение их к единому регистру

def unique_words(name, dataframe):
    print('Определение уникальных значений')
    print(dataframe[name].value_counts())
    dataframe[name] = dataframe[name].str.lower()
    print('Проверка результатов')
    print(dataframe[name].value_counts())
    return


Количество полностью дублирующихся строк в таблице: 54


In [15]:
unique_words('education', clients_solvency)
unique_words('gender', clients_solvency)

Определение уникальных значений
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64
Проверка результатов
среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64
Определение уникальных значений
F      14236
M       7288
XNA        1
Name: gender, dtype: int64
Проверка результатов
f      14236
m       7288
xna        1
Name: gender, dtype: int64


In [16]:
unique_words('family_status', clients_solvency)

Определение уникальных значений
женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
Проверка результатов
женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


In [17]:
unique_words('income_type', clients_solvency)

Определение уникальных значений
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64
Проверка результатов
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64


In [18]:
unique_words('purpose', clients_solvency)

Определение уникальных значений
свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
за

In [19]:
clients_solvency['family_status'] = clients_solvency['family_status'].replace('гражданский брак', 'сожительство')
clients_solvency.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,unemployment_code
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,0
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,0
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,0
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,0
4,0,0,53,среднее,1,сожительство,1,f,пенсионер,0,158616,сыграть свадьбу,340266


Для определения дубликатов сначала мы применяем метод .duplicated().sum() ко всей таблице - это позволит нам оценить количество полностью повторяющихся строк. Таких строк в таблице не обнаружено.
Далее мы составили функцию последовательно запускающую поиск уникальных значений, приведение всех значений к нижнему регистру и печать нового поиска- это позволило нам избавиться от повторов названий категорий, написанных в буквах различных регистров. Такого рода ошибки могут возникнуть, как и при внесении данных различными операторами, так и при сведении нескольких таблиц в одну и сканировании документов. После выполнения функции unique_words вы таблице остались только слова, отличающиеся своей формой, которые необходимо удалить при помощи лемматизации. 

### 2.1.5.2. Корректировка данных с удалением возникших дубликатов

In [20]:
# Удаление возникших дубликатов
clients_solvency = clients_solvency.drop_duplicates().reset_index(drop=True)
# Проверка удаления дубликатов и проверка индексов
print('Количество полностью дублирующихся строк в таблице:', clients_solvency.duplicated().sum())
clients_solvency.head()

Количество полностью дублирующихся строк в таблице: 0


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,unemployment_code
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,0
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,0
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,0
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,0
4,0,0,53,среднее,1,сожительство,1,f,пенсионер,0,158616,сыграть свадьбу,340266


В связи с возникшими после заполнения пропущенных значений дубликатами - была осуществлена операция удаления дубликатов при помощи функции drop_duplicates(). После этого осуществлена проверка правильности выполнения операции. Дубликаы удалены, индексы упорядочены

<a id='the_destination11'></a>
## 2.1.6. Лемматизация

In [21]:
from pymystem3 import Mystem
m = Mystem()
import collections

counter_word = collections.Counter()
#печать списка уникальных значений столбца purpose
print('Список всех целей кредита')
print(clients_solvency['purpose'].value_counts())

Список всех целей кредита
свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться

In [22]:
#создание массива лемматизированных значений столбца с целями кредита purpose
# функция создания массива лемм
def lemmas_array_create(row):
    global counter_word
    lemmas_array = []
    lemmas_total = []
    lemma = m.lemmatize(row)
# счетчик отдельных слов-лемм
    for element in lemma:
        counter_word[element] += 1
#объединение в одну строку
        lemmas_array.append(''.join(lemma))
        return lemmas_array
    
lemmas_array = clients_solvency['purpose'].apply(lemmas_array_create)
print('Печать первых трех элементов массива лемм')
print(lemmas_array[:3])
print('Cчетчик отдельных лемм')
print(counter_word)

Печать первых трех элементов массива лемм
0              [покупка жилье\n]
1    [приобретение автомобиль\n]
2              [покупка жилье\n]
Name: purpose, dtype: object
Cчетчик отдельных лемм
Counter({'покупка': 4443, 'операция': 2604, 'на': 2222, 'строительство': 1878, 'получение': 1314, 'автомобиль': 972, 'сделка': 941, 'заниматься': 904, 'свадьба': 791, 'сыграть': 765, 'жилье': 646, 'недвижимость': 633, 'ремонт': 607, 'свой': 478, 'приобретение': 461, 'дополнительный': 460, 'высокий': 452, 'образование': 447, 'профильный': 436})


In [23]:
# создание функции категоризации целей кредита
def replace_name(array, p_array):
    for element in array:
        if 'свадьба' in str(element):
            purpose_array.append('оплата расходов на свадьбу')
        if 'автомобиль' in str(element):
            purpose_array.append('покупка автомобиля')
        if 'образование' in str(element):
            purpose_array.append('оплата обучения')
        if 'недвижимость' in str(element):
            if 'операции' in str(element):
                purpose_array.append('операции с недвижимостью')
            if 'коммерческий' in str(element):
                purpose_array.append('покупка коммерческой недвижимости')
            if 'строительство' in str(element):
                purpose_array.append('строительство недвижимости')
            else:
                purpose_array.append('покупка жилой недвижимости')
        if 'жилье' in str(element):
            if 'операции' in str(element):
                purpose_array.append('операции с недвижимостью')
            else:
                purpose_array.append('покупка жилой недвижимости')
        if 'ремонт'in str(element):
            purpose_array.append('ремонт')
    return

Для визуального представления количества разных лемм-слов был введен счетчик-лемм через функцию Counter библиотеки collections. На выведенном списке лемм и их частоты видно, что наиболее часто употребляются слова недвижимость, покупка, жилье, однако, так как операции с недвижимостью нельзя ограничить только покупкой и не стоит объединять покупку коммерческой недвижимости и покупку жилой недвижимости, то в данном случае целесообразно остановиться на анализе словосочетаний. Возможно ввести дополнительную функцию поиска и подсчета словосочетаний, например "коммерческий" + "недвижимость", однако, так как список в данном случае не очень длинный, то можно остановиться на визуальном анализе.

In [24]:
purpose_array = []
replace_name(lemmas_array, purpose_array)
print('Печать массива после обработки')
purpose_array[:10]

Печать массива после обработки


['покупка жилой недвижимости',
 'покупка автомобиля',
 'покупка жилой недвижимости',
 'оплата обучения',
 'оплата расходов на свадьбу',
 'покупка жилой недвижимости',
 'покупка жилой недвижимости',
 'оплата обучения',
 'оплата расходов на свадьбу',
 'покупка жилой недвижимости']

Для обработки данных по целям кредита необходимо сократить количество разнообразных целей кредита и привести их к однотипному виду, это осуществляется при помощи метода лемматизации, импортируемого из библиотеки pymystem3. Применение этого метода позволяет выделить слова в именительном падеже, которые будут более удобно для дальнейшей обработки и категоризации. Для начала при помощи метода value.counts() мы оцениваем количество разнообразных целей. Далее было принято решение создать функцию осуществляющую лемматизацию информации о целях кредитов, где лемматизированные предложения записываются в отдельный архив для удобства дальнейшей обработки. После осуществления лемматизации, цели стали удобны для дальнейшей обработки, выделение стеммера в данном случае нецелесообразно, так как приведет к сложностям при анализе различий между целями, так как в тексте присутствует жилье и жилой в целях разного назначения. 
Для приведения целей кредитов к одинаковым формулировкам была создана функция, которая на основании значимых слов выбирала соответствующую по формулировке цель кредита и добавляла ее в массив целей. 
При обработке данных по целям кредитов те кредиты на недвижимость, где не указан был тип недвижимости были переведены в раздел "покупка жилой недвижимости", это связано с тем, что на современном российском рынке объем кредитования на жилье превышает объем кредитования на развитие бизнеса в почти в 5 раз (по информации банка Хоум кредит), так что наиболее вероятно, что данные средства были потрачены на покупку собственного жилья. Объединение этих категорий в одну - на покупку недвижимости нецелесообразно, так как зачастую люди, приобретающие коммерческую недвижимость являются людьми с более высоким уровнем доходов и это приведет к искажению статистики при конечном анализе данных. Однако цели с указанием "операции с недвижимостью" были объединены в одну категорию, в связи с тем, что эти цели более близки по своей структуре. Несмотря на то, что категория ремонт присутствует в таблице в единственном числе убирать ее в категорию "другое" недопустимо, так как в исследованиях проводимых экспертами (https://www.banki.ru/news/lenta/?id=10344304) ремонт являлся наиболее часто встречающейся причиной взятия кредита. 

<a id='the_destination12'></a>
## 2.1.7.  Категоризация данных

In [ ]:
clients_solvency_data = clients_solvency. loc[:, ['children', 'family_status', 'debt', 'total_income', 'gender']]
clients_solvency_additional_info = clients_solvency.loc[:, [ 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'gender', 'income_type', 'debt', 'purpose', 'unemployment_code']]
#добавление столбца с значениями массива 
for i in range(len(purpose_array)):
    clients_solvency_data.loc[i, 'purpose_updated'] = purpose_array[i]

In [ ]:
# Выделений категорий дохода при помощи квантилей
print('Квантили уровня дохода', clients_solvency_data['total_income'].quantile(q=[0.2,0.4,0.6,0.8]))

# введение новых категорий уровней дохода, каждый соответствует 20% клиентов
def assesment_income_category(income):
    if income < 100000:
        return 'доход ниже среднего'
    if 100000 <= income < 130000:
        return 'доход средний'
    if 130000 <= income < 160000:
        return 'доход выше среднего'
    if 160000 <= income < 210000:
        return 'доход высокий'
    if income >= 210000:
        return 'доход сверхвысокий'
    
clients_solvency_data['income_level'] = clients_solvency_data['total_income'].apply(assesment_income_category)
clients_solvency_data.head()

In [ ]:
# введение уровней размера семьи
def family_size_meter(number):
    if number == 0:
        return 'бездетная семья'
    if 0 < number <= 2:
        return 'малодетная семья'
    if 3 <= number <= 4:
        return 'среднедетная семья'
    if number > 4:
        return 'многодетная семья'

clients_solvency_data['family_size'] = clients_solvency_data['children'].apply(family_size_meter)

clients_solvency_data.head(10)

Для анализа данных по платежеспособности клиентов в рамках данного задания нам необходимо проанализировать лишь некоторые из параметров, поэтому мы осуществим разбиение таблицы на основную таблицу с данными необходимыми для анализа - clients_solvency_data и вторую таблицу - clients_solvency_additional_info.
В таблицу с информацией, необходимой для анализа добавляем столбец с скорректированными наименованиями целей кредитования purpose_updated.
Для категоризации уровня доходов населения вводим функцию assesment_income_category. 
При выделении уровней доходов можно ориентироватьсяя на группы, выделенные Федеральной службой госстатистики (https://avatars.mds.yandex.net/get-zen_doc/48747/pub_5c58a8cc4c240600b03fdd70_5c58aa7c18708200aaf5cdfd/scale_2400), объединив группы крайняя нищета, нищета и бедность -  в категорию "очень низкий уровень дохода", оставив в качестве верхней границы верхнюю границу уровня бедности - 20 тр, "низкий уровень дохода" соотнесем с группой выше бедности - от 20 до 30 тр, "средний уровень дохода" сопоставим среднему достатку - от 30 до 60 тр, "уровень дохода выше среднего" соотнесем с уровнем богатые - от 60 до 90 тр, "высокий доход" - с группой богатые от 90 до 150 тр, и "очень высокий уровень дохода" соотнесем с группой сверхбогатые- выше 150 тр. Однако, введение таких категорий приведет к тому, что большинство клиентов попадет в группу с высоким и сверхвысоким уровнем дохода, поэтому при выделении групп будем ориентироваться на результаты определения квантилей дохода - то есть уровней дохода, соответствующих 20, 40, 60 и 80% клиентов. 
Для улучшения понимания связи платежеспособности клиентов с наличием детей в семье вводим функцию family_size_meter. Для разделения по категориям воспользуемся категориями, приводимые в учебных пособиях по социологии - бездетные семьи, малодетные семьи (1-2 ребенка), среднедетные семьи (3-4 ребенка) и многодетные семьи. 

<a id='the_destination13'></a>
# 3. Изучение влияния различных факторов на факт возврата кредита в срок

<a id='the_destination14'></a>
## 3.1 Изучение влияния наличия детей

In [ ]:
family_solvency = clients_solvency_data.groupby(by = 'family_size')['debt']
print('Процент должников среди клиентов с различным количеством детей')
print(family_solvency.sum()* 100 /family_solvency.count())

In [ ]:
family_solvency_pivot = clients_solvency_data.pivot_table(index = ['children', 'income_level'], values='debt', aggfunc={'debt': ['count', 'sum', 'mean']})
display(family_solvency_pivot)

In [ ]:
family_purposes_solvency_pivot = clients_solvency_data.pivot_table(index = ['family_size', 'purpose_updated'], values='debt', aggfunc={'debt': ['count', 'sum', 'mean']})
display(family_purposes_solvency_pivot)

Сгруппировав данные по категориям, учитывающим количество детей, хорошо видно, что среди малодетных семей должников незначительно больше, чем среди бездетных и среднедетных. Среди многодетных семей должников нет, по-видимому эта категория семей редко берут кредиты и в случае, если все же берут их способны их закрывать в срок. Что касается увеличения числа должников среди малодетных семей по сравнению с бездетными видимо это связано, что при значительно большей финансовой нагрузке такие семьи вынуждены тратить больше на расходы и не в состоянии рассчитывать свои финансовые возможности. Как хорошо видно, среднедетные семьи находятся на уровне бездетных семей, что видимо говорит о том, что если семьи с 3-4 детьми решаются взять кредит, то лучше рассчитывают свои возможности и при том же среднем уровне доходов, что и малодетные семьи лучше справляются с распределением ресурсов. 
В итоговой таблице была выведен зависимость доли должников от количества детей с учетом уровня дохода. По данным выявляется интересная закономерность, что если для бездетных клиентов доля должников примерно одинакова для всех уровней дохода (резкий скачок на низких уровнях дохода не показателен из-за малой выборки, менее 100 человек), то для малодетных семей меньше должников при средних уровнях дохода, чем на высоких (для семей с лвумя летьми средний уровень дохода приходится выкинуть из-за малой выборки), то есть при высоких доходах становится сложнее распределять ресурсы и оценивать свои расходы.
Так как выборка для многодетных семей незначительно, можно отметить закономерность, что для них все же скорее наблюдается обратная закономерность - должников при высоких уровнях дохода меньше. 
Интересно еще изучить таблицу, где приведена доля должников для семей разного размера и цели кредитов. Среди бездетных семей наименьшее количество должников среди тех, кто брал кредит на строительство, а наибольшее - среди тех, кто покупал коммерческую недвижимость, у малодетных семей наименьшее количество должников среди тех, кто брал кредит на ремонт, а наибольшее - также среди тех, кто покупал коммерческую недвижимость, для среднедетных семей наименьшее количество должников среди тех, кто брал кредит на покупку автомобиля, а наибольшее для тех, кто брал кредит на строительство. Наибольшая доля должников среди тех, у кого 1-2 ребенка и кто брал кредит на покупку коммерческой недвижимости 10,7%, а наименьшая для среднедетной семьи, покупающей автомобиль - 3,5%.
В целом, необходимо отметить, что несмотря на разную динамику в разных группах клиентов - наличие и количество детей незначительно сказывается на платежеспособность клиентов и в среднем по группе колебания не превышают 2%. 

<a id='the_destination15'></a>
## 3.2 Влияние семейного положения

In [ ]:
marriage_solvency = clients_solvency_data.groupby(by='family_status')['debt']
print('Процент должников среди клиентов с различным семейным статусом')
print(marriage_solvency.sum()* 100 /marriage_solvency.count())

In [ ]:
marriage_solvency_pivot = clients_solvency_data.pivot_table(index = ['family_status', 'income_level'], values='debt', aggfunc={'debt': ['count', 'sum', 'mean']})
display(marriage_solvency_pivot)

Сгруппировав данные по категориям, учитывающим семейный статус, хорошо видно, что среди неженатых клиентов доля должников выше, чем среди замужник и вдовых клиентов. Несмотря на то, что доход вдовых клиентов ниже, чем у остальных категорий, доля должников среди них ниже, чем у остальных категорий. Разведенные и замужние клиенты имеют примерно одинаковую долю должников. Можно сказать, что не смотря на одинаковый уровень доходов с другими категориями клиентов но неженатые и живущие без официальной регистрации клиенты в большей степени склонны к образованию долгов перед банком. 
В итоговой таблице была выведена зависимость доли должников от семейного статуса и уровня доходов. Можно отметить, что доля должников выше среди сожительствующих и неженатых с высоким уровнем дохода. Для женатых, вдовцов и разведенных наибольшее количество должников среди людей с уровнем дохода выше среднего. В целом, для разных клиентов с разным семейным статусом доля должников отличается не так значительно - в среднем на 2-3%, наименьшее количество должников среди вдовцов с уровнем дохода ниже среднего (4,6%), наибольшее - среди живущих в незарегистрированных отношениях с средним доходом (11%)

<a id='the_destination16'></a>
## 3.3. Влияние уровня дохода

In [ ]:
income_solvency = clients_solvency_data.groupby(by ='income_level')['debt']
print('Процент должников среди клиентов с различным уровнем дохода')
print(income_solvency.sum()* 100 /income_solvency.count())

In [ ]:
income_solvency_pivot = clients_solvency_data.pivot_table(index = ['income_level', 'gender'], values='debt', aggfunc={'debt': ['count', 'sum', 'mean']})
display(income_solvency_pivot)

После группировки данных по уровням дохода, можно заметить, что наибольшее количество должников среди клиентов банка с доходом выше среднего (8,89%). Самый низкий уровень должников среди клиентов с сверхвысоким уровнем дохода (6,995%). В итоговой таблице для анализа также добавлен столбец с полом клиента, хорошо видно, что на всех уровнях дохода мужчин-должников больше чем женщин (6-7% у женщин и 10-11% у мужчин), однако сам уровень дохода влияет не так значительно, среди женщин разных уровней дохода и мужчин доля должников колеблется незначительно. Однако в целом в среднем доля должников между различными категориями доходов меняется незначительно - в пределах 1-2%. Самый низкий уровень должников среди женщин с сверхвысоким доходом (6.1%), а самый высокий - среди мужчин с средним доходом (11,7%).

<a id='the_destination17'></a>
## 3.4 Влияние целей

In [ ]:
purpose_solvency = clients_solvency_data.groupby(by='purpose_updated')['debt']
print('Процент должников среди клиентов с различными целями кредитования')
print(purpose_solvency.sum()* 100 /purpose_solvency.count())

In [ ]:
purpose_solvency_pivot = clients_solvency_data.pivot_table(index = ['purpose_updated', 'income_level'], values='debt', aggfunc={'debt': ['count', 'sum', 'mean']})
display(purpose_solvency_pivot)

При анализе таблицы можно отметить, что доля должников при покупке автомобиля для людей с высоким доходом также выше, чем для людей с средним доходом, это может быть связано с покупкой автомобилей более дорогой ценовой категории. Надо отметить, что в отличие от предыдущего анализа, с учетом новой категоризации, самая платежеспособная категория - люди с свервысоким уровнем дохода, но среди этой группы самые высокие уровни должников в категориях покупка и строительство жилой недвижимости, видимо это связано что для этой категории характерна покупка более дорогой недвижимости и строительство более масштабных проектовю Самая низкая доля должников среди тех кто брал кредит на ремонт среди группы людей с сверхвысоких доходов (6%), а наибольший среди тех, кто брал кредит на покупку коммерческой недвижимости (10,3%) и оплату свадьбы (10,7%) среди людей с высоким уровнем дохода. Надо отметить, что для людей с высокими уровнями доходов наибольшее количество должников среди тех кто брал кредит на свадьбу, покупку коммерческой недвижимости, покупку автомобиля. Хорошо видно, что при покупке жилой недвижимости почти все уровни дохода удерживают примерно одинаковый процент неплательщиков - около 8%, однако люди с низким уровнем дохода испытывают больше трудностей при выплачивании кредита на строительство недвижимости и обучение, вероятно это связано с тем, что расходы в данных категориях могут превышать запланированные. В целом, цели кредитования значительно влияют на платежеспособность клиентов, но их невозможно оценивать без оценки объема кредитных средств (является ли покупаемая вещь вещью премиум класса или нет), вероятнее всего, что люди с высоким доходом покупке вещей премиум класса, таких как машины, организация свадьбы, склонны переплачивать за товар и таким образом переоценивать свою платежеспособность. Покупка коммерческой недвижимости является наиболее рискованной группой для кредитования вероятно в связи с тем, что при ее покупке клиент учитывает потенциальные доходы от этой недвижимости.

<a id='the_destination18'></a>
# 4. Общий вывод

Платежеспособность клиентов банка так или иначе зависит от всех вышеперечисленных факторов - количества детей, семейного положения, уровня доходов и целей, на которые планируется потратить кредит. Наибольшее влияние на долю должников среди клиентов банка оказывают семейный статус клиентов и цели клиентов, однако, как показал анализ рассматривать цели без учета доходов клиентов является неверным принципом, так как уровень доходов оказывает влияние на выбор цели кредитования и размера кредита. В целом влияние количества детей незначительно - не более 1-2%, более значительное влияние на количество должников оказывает не входящий в анализ параметр, который мы оценили в рамках дополнительного исследования - пол кредитуемого, как показали исследования пол клиента оказывает более существенное влияние, чем семейное состояние и должен учитываться при проведении анализов.

В целом, изучив предоставленные данные можно сделать вывод, что мужчины значительно более рискованная для кредитования группа клиентов (среди них должников на 5% больше, чем среди женщин), люди с выше среднего и  высоким уровнями дохода (от 130 до 210 тр) являются более рискованными клиентами, особенно если берут кредит на покупку коммерческой недвижимости, организацию свадьбу, покупку автомобиля и сроительство недвижимости (среди них может быть на 4% больше должником, чем среди людей с сверхвысокими доходами и на 2% больше по сравнению с людьми с низкими и средними доходами). Количество детей на платежеспособность влияет незначительно, однако, многодетные и бездетные клиенты в целом имеют меньший процент должников (на 1-2%). Неженатые и живущие в незарегистрированных отношениях клиенты с высоким и выше среднего доходом имеют больших процент должников, по сравнению с женатыми, вдовцами или разведенными клиентами (разница составляет до 3-4% на высоком уровне дохода).
Наименее рискованный клиент - это женщина с сверхвысоким доходом, состоящая в зарегистрированном браке, берущая кредит на ремонт или свадьбу. Наиболее рискованный - мужчина с уровнем дохода выше среднего (130-160 тр), живующий в незарегистрированных отношениях, берущий кредит на покупку коммерческой недвижимости или свадьбу.